# Proyecto Final - Deep Learning y Redes Neuronales

MIIA4406 - 201819 - Universidad de los Andes
### Integrantes:
- Jorge Eduardo Rodriguez Cardozo - 200711501
- German Augusto Carvajal Murcia -  201313516

In [1]:
import os
import pandas as pd
import numpy as np
from time import time
inicio=time()
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import roc_auc_score,make_scorer,accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from keras.preprocessing.image import img_to_array, load_img
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import string
import unidecode
nltk.download('wordnet')
nltk.download('stopwords')
%matplotlib inline
path = os.getcwd()
dataTraining = pd.read_csv(os.path.join(path, 'data', 'dataTraining.csv'), encoding='UTF-8', index_col=0)

Using TensorFlow backend.


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/germancarvajal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/germancarvajal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    import sys
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    sys.stdout.write('\r %s |%s| %s%% %s' % (prefix, bar, percent, suffix))
    sys.stdout.flush()
    # Print New Line on Complete
    if iteration == total: 
        print()

In [3]:
Img_size=(256,160) #Size of the images to import
Gray=True #True if the images to load are in grayscale, False for RGB encoding
words= 2000 #Number of text features to consider in the algorithm
usePCA = (False, 1000) #Arg1: True for using PCA over the images data for dimensionality reduction, Arg2: Number of principal components to use

In [4]:
wordnet_lemmatizer = WordNetLemmatizer()
def split_into_lemmas(text):
    text = unidecode.unidecode(text.lower())
    words = text.split()
    nopunc = [char for char in words if char not in string.punctuation]
    clean_mess = [word for word in nopunc if word not in stopwords.words('english')]
    return [wordnet_lemmatizer.lemmatize(word).encode('ascii') for word in clean_mess]

In [5]:
vect = CountVectorizer(ngram_range=(1, 2), max_features=words, analyzer=split_into_lemmas,binary=False)
X_txt = vect.fit_transform(dataTraining['plot'])
print('Text input size: ',X_txt.shape)

Text input size:  (7895, 2000)


In [6]:
X_image=np.empty((dataTraining.shape[0],(Img_size[0]*Img_size[1]*(3-2*Gray))))
N=dataTraining.shape[0]
n=0
for i in dataTraining.index:
    img = load_img(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg'),target_size=Img_size,grayscale=Gray)  # this is a PIL image
    x = img_to_array(img).flatten()  # this is a Numpy array with shape (Img_size[0]*Img_size[1]*(3-2*I(Gray)))
    x = x.reshape((1,) + x.shape)
    X_image[n]=x
    n=n+1
    printProgressBar(n,N,prefix = 'Progress:', suffix = 'Complete', length = 50)

 Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [7]:
if usePCA[0]:
    pca=PCA()
    pca.fit(X_image)
    plt.figure(1, figsize=(10, 5))
    plt.xlabel('Num factors')
    plt.ylabel('Explained variance')
    plt.axvline(x=usePCA[1],color='red')
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    pcaT=PCA(usePCA[1])
    X_image=pcaT.fit_transform(X_image)
else:
    print('No PCA')

No PCA


In [8]:
X=np.concatenate((X_txt.toarray(),X_image),axis=1)
Y_lab=dataTraining['genres']
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(pd.Series(Y_lab).map(lambda x: eval(x)))
print('Features: ',X.shape)
print('Labels: ',Y.shape)

Features:  (7895, 42960)
Labels:  (7895, 24)


In [9]:
X_int,X_test,Y_int,Y_test=train_test_split(X,Y,test_size=0.1,random_state=123)
Index_train,Index_validation=train_test_split(range(X_int.shape[0]),test_size=0.2/0.9,random_state=0)

In [10]:
print('Trainig: ',X_int[Index_train].shape,Y_int[Index_train].shape)
print('Validation: ',X_int[Index_validation].shape,Y_int[Index_validation].shape)
print('Test: ',X_test.shape,Y_test.shape)

Trainig:  (5526, 42960) (5526, 24)
Validation:  (1579, 42960) (1579, 24)
Test:  (790, 42960) (790, 24)


In [11]:
pd.DataFrame(np.transpose([mlb.classes_,Y_int[Index_train].sum(axis=0),Y_int[Index_validation].sum(axis=0),Y_test.sum(axis=0)]),columns=['Class','Train','Validation','Test'])

,Class,Train,Validation,Test
0,Action,884,283,136
1,Adventure,705,216,103
2,Animation,182,54,24
3,Biography,248,90,35
4,Comedy,2166,591,289
5,Crime,1019,279,149
6,Documentary,293,90,36
7,Drama,2793,784,388
8,Family,489,129,64
9,Fantasy,500,127,80


In [12]:
validation_fold=np.empty(X_int.shape[0])

In [13]:
for i in Index_train:
    validation_fold[i]=-1
for i in Index_validation:
    validation_fold[i]=0

In [14]:
from sklearn.model_selection import PredefinedSplit
ps = PredefinedSplit(validation_fold)

In [15]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation AUC: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

#Scorer
my_roc=make_scorer(roc_auc_score,greater_is_better=True,needs_proba=True,average='macro')

In [16]:
# build a classifier
clf1 = OneVsRestClassifier(RandomForestClassifier())

# specify parameters and distributions to sample from
param_dist = {"estimator__n_estimators": sp_randint(10, 200),
              "estimator__max_depth": sp_randint(3, 10),
              "estimator__max_features": sp_randint(1, 11),
              "estimator__bootstrap": [True, False],
              "estimator__criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search=10
random_search1 = RandomizedSearchCV(clf1, param_distributions=param_dist,cv=ps,n_iter=n_iter_search,return_train_score=True,verbose=1,scoring=my_roc,random_state=123)

#Report results
start = time()
random_search1.fit(X_int, Y_int)
print("RandomizedSearchCV for RandomForest took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search1.cv_results_)

Fitting 1 folds for each of 10 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 40.9min finished


RandomizedSearchCV for RandomForest took 2738.29 seconds for 10 candidates parameter settings.
Model with rank: 1
Mean validation AUC: 0.632 (std: 0.000)
Parameters: {'estimator__bootstrap': False, 'estimator__criterion': 'entropy', 'estimator__max_depth': 6, 'estimator__max_features': 3, 'estimator__n_estimators': 94}

Model with rank: 2
Mean validation AUC: 0.626 (std: 0.000)
Parameters: {'estimator__bootstrap': False, 'estimator__criterion': 'gini', 'estimator__max_depth': 7, 'estimator__max_features': 9, 'estimator__n_estimators': 199}

Model with rank: 3
Mean validation AUC: 0.614 (std: 0.000)
Parameters: {'estimator__bootstrap': True, 'estimator__criterion': 'entropy', 'estimator__max_depth': 6, 'estimator__max_features': 10, 'estimator__n_estimators': 106}



In [17]:
clf1=OneVsRestClassifier(RandomForestClassifier(bootstrap=random_search1.best_params_['estimator__bootstrap'],criterion=random_search1.best_params_['estimator__criterion'],
                                               max_depth=random_search1.best_params_['estimator__max_depth'],max_features=random_search1.best_params_['estimator__max_features'],
                                               n_estimators=random_search1.best_params_['estimator__n_estimators']))
clf1.fit(X_int,Y_int)
Y_pred=clf1.predict_proba(X_test)
roc_auc_score(Y_test,Y_pred)

0.62089490387799839

In [18]:
dataTesting = pd.read_csv(os.path.join(path, 'data', 'dataTesting.csv'), encoding='UTF-8', index_col=0)
X_image_pred=np.empty((dataTesting.shape[0],(Img_size[0]*Img_size[1]*(3-2*Gray))))
N=dataTesting.shape[0]
n=0
for i in dataTesting.index:
    img = load_img(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg'),target_size=Img_size,grayscale=Gray)  # this is a PIL image
    x = img_to_array(img).flatten()  # this is a Numpy array with shape (Img_size[0]*Img_size[1]*(3-2*I(Gray)))
    x = x.reshape((1,) + x.shape)
    X_image_pred[n]=x
    n=n+1
    printProgressBar(n,N,prefix = 'Progress:', suffix = 'Complete', length = 50)
X_txt_pred = vect.transform(dataTesting['plot'])
if usePCA[0]:
    X_image_pred=pcaT.transform(X_image_pred)
X_pred=np.concatenate((X_txt_pred.toarray(),X_image_pred),axis=1)

 Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [19]:
pred1=pd.DataFrame(clf1.predict_proba(X_pred),columns=['p_'+s for s in mlb.classes_],index=dataTesting.index)
pred1.to_csv(path_or_buf ='OvsR_RF_Split_Calib.csv',index_label='ID',encoding='UTF-8')

In [20]:
print("Total execution time took %.2f seconds" % ((time() - inicio)))

Total execution time took 3901.89 seconds
